## EDA & Visualization

> Still WIP(Work in Progress)

## Import Library & Dataset

In [ ]:
!pip install seaborn==0.11
import seaborn as sns

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


from matplotlib.offsetbox import AnchoredText
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv')

train.describe()

## Categorical Feature

One point of this competition is to encode the categorical value again, unlike the previous competition.

In [ ]:
fig, ax = plt.subplots(3, 4, figsize=(15, 12))
fig.set_facecolor('#d0d0d0') 

for i in range(10):
    # count 
    cnt_tmp = train[f'cat{i}'].value_counts().sort_index()
    ax[i%3][i//3].bar(cnt_tmp.index, cnt_tmp, color='#244747', label='Count')
    ax[i%3][i//3].set_yticks([])
    ax[i%3][i//3].margins(0.05, 0.2)
    
    # target
    target_tmp = train.groupby(f'cat{i}').mean()['target']
    ax2=ax[i%3][i//3].twinx()
    ax2.set_yticks([])
    ax2.bar(target_tmp.index, target_tmp, color='#d4dddd', alpha=0.7, label='Mean of Target')
    ax2.set_ylim(7, 8.4)
    
    divider = make_axes_locatable(ax[i%3][i//3])
    cax = divider.append_axes("top", size="8%", pad=0)
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.set_facecolor('black')

    at = AnchoredText(f'cat{i}', loc=10, 
                      prop=dict(backgroundcolor='black',
                                size=10, color='white', weight='bold'))
    cax.add_artist(at)
    
ax[1][3].set_visible(False)
ax[2][3].set_visible(False)

fig.text(0.018, 1.03, 'Categorical Feature Distribution', fontsize=20, fontweight='bold', fontfamily='serif')
fig.text(0.018, 1, 'Gray is mean of target', fontsize=15, fontweight='light', fontfamily='serif')



plt.tight_layout()
plt.show()

## Categorical Value Relation Heatmap


some combination between `cat6`~`cat9` have **absolutly** lower value(mean of target).

In [ ]:
fig = plt.figure(figsize=(30, 30))
gs = fig.add_gridspec(sum([len(train[f'cat{i}'].unique()) for i in range(1, 10)]), 
                     sum([len(train[f'cat{i}'].unique()) for i in range(10)]))

idx = np.cumsum([0] + [len(train[f'cat{i}'].unique()) for i in range(10)])

         
for i in range(10):
    for j in range(10):
        if i <= j : 
            continue
        pivot_tmp = pd.pivot_table(train, values='target', index=[f'cat{i}'], columns=[f'cat{j}'], aggfunc=np.mean, fill_value=0)
        ax = fig.add_subplot(gs[idx[i]-2:idx[i+1]-2, idx[j]:idx[j+1]])
        
        sns.heatmap(pivot_tmp, 
                    square=True, 
                    center=7.456, # mean of target
                    cbar=False, 
                    cmap=sns.diverging_palette(240, 10),
                    xticklabels=(i==9), yticklabels=(j==0),
                    ax=ax)

        if j : ax.set_ylabel('')    
        if i<9 : ax.set_xlabel('')

fig.text(0.125, 0.89, 'Categorical Feature Relation Heatmap', fontsize=20, fontweight='bold', fontfamily='serif')            

plt.show()

## Continuous 



Obviously, continuous variables also have a bias according to their central distribution.

For visualization, sample and visualize some.

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(17, 12), sharex=True)
fig.set_facecolor('#d0d0d0') 

for i in range(14): 
    sns.kdeplot(data=train, x=f'cont{i}', 
                fill=True,
                linewidth=0,
                color='#244747', alpha=1,
                ax=ax[i%3][i//3])
    ax[i%3][i//3].set_yticks([])
    ax[i%3][i//3].set_xlabel('',visible=False)
    ax[i%3][i//3].margins(0.05, 0.2)
    
    # dviider
    divider = make_axes_locatable(ax[i%3][i//3])
    cax = divider.append_axes("top", size="8%", pad=0)
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.set_facecolor('black')

    at = AnchoredText(f'cont{i}', loc=10, 
                      prop=dict(backgroundcolor='black',
                                size=10, color='white', weight='bold'))
    cax.add_artist(at)
    
ax[-1][-1].set_visible(False)
fig.text(0.018, 1.03, 'Continuous Feature Density Plot', fontsize=20, fontweight='bold', fontfamily='serif')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
fig.set_facecolor('#d0d0d0') 
corr = train.drop('id', axis=1).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(corr, 
            square=True, 
            linewidth=0.2,
            cbar=False,
            mask=mask,
            annot=True,
            center=0,
            cmap=sns.diverging_palette(240, 10),
            ax=ax)

fig.text(0.075, 1, 'Correlation: Continuous Feature & Target', fontweight='bold', fontfamily='serif', fontsize=15) 
ax.set_xticklabels(ax.get_xticklabels(), fontfamily='serif', rotation=90, fontsize=11)
ax.set_yticklabels(ax.get_yticklabels(), fontfamily='serif', rotation = 0, fontsize=11)
plt.tight_layout()
plt.show()

For relation, check i use `pariplot`

Sampling is nice choices for Large data

In [ ]:
sns.pairplot(train.iloc[:,11:].sample(10000), kind="hist", corner=True)